In [ ]:
import os
import random
import string
import time
from flask import Flask, request, jsonify
from google.cloud import storage, speech
from google.oauth2 import service_account
import googleapiclient.discovery
import torchaudio
from googletrans import Translator
import json
app = Flask(__name__)

# Path to the service account JSON file
client_file = "sa.json"
credentials = service_account.Credentials.from_service_account_file(client_file)
service = googleapiclient.discovery.build('speech', 'v1', credentials=credentials)
storage_client = storage.Client.from_service_account_json(client_file)


import requests

def generate_model_answer(prompt):
    # Specify the URL for the model endpoint
    url = "https://us-central1-aiplatform.googleapis.com/v1/projects/nitthack/locations/us-central1/publishers/google/models/gemini-1.0-pro:streamGenerateContent"
    # Send a POST request with the JSON data
    headers = {
        "Authorization": os.process.GOOGLE_AUTH,  # Replace with your actual access token
        "Content-Type": "application/json; charset=utf-8"
    }
    response = requests.post(url, headers=headers, data=prompt)

    # Check if the request was successful
    if response.status_code == 200:
        return response.json()
    else:
        return response

# Helper function to translate a word
def translate_word(word: str, source_lang: str, target_lang: str = 'en'):
    from deep_translator import GoogleTranslator
    res = GoogleTranslator(source=source_lang, target=target_lang).translate(word) 
    return res


# Endpoint to handle audio file upload and transcription
@app.route("/upload_and_transcribe/", methods=["POST"])
def upload_and_transcribe():
    audio_file = request.files["audio_file"]
    gcs_blob = ''.join(random.choices(string.ascii_lowercase, k=5))
    audio_file.save(f"{gcs_blob}.wav")
    r = torchaudio.info(f"{gcs_blob}.wav")
    bucket = storage_client.get_bucket('speechstoragebucketsamsung')
    blob = bucket.blob(gcs_blob)
    blob.upload_from_filename(f"{gcs_blob}.wav", content_type="audio/wav")
    uri = f'gs://speechstoragebucketsamsung/{gcs_blob}'

    config = {
        "encoding": "LINEAR16",
        "languageCode": "en-US",
        "enableWordTimeOffsets": True,
        "sampleRateHertz" : r.sample_rate,
        "audioChannelCount" : r.num_channels,
    }

    body = {
        "config": config,
        "audio": {
            "uri": uri
        }
    }

    service_request = service.speech().longrunningrecognize(body=body)
    operation = service_request.execute()

    name = operation['name']
    service_request = service.operations().get(name=name)
    while True:
        print('Waiting for server processing...')
        time.sleep(1)
        response = service_request.execute()
        if 'done' in response and response['done']:
            break

    transcriptions = []
    for result in response['response']['results']:
        transcriptions.append(result['alternatives'][0]['transcript'])

    return jsonify(transcriptions)


@app.route("/translate/", methods=["POST"])
def translate_word_endpoint():
    data = request.json
    if not data:
        return jsonify({"error": "No JSON data provided"}), 400
    
    word = data.get("word")
    source_lang = data.get("source_lang")
    target_lang = data.get("target_lang", "en")
    
    if not word or not source_lang:
        return jsonify({"error": "Missing required fields: 'word' and 'source_lang'"}), 400

    translated_word = translate_word(word, source_lang, target_lang)
    return jsonify({"translated_word": translated_word})

@app.route("/generate_flashcards", methods=["POST"])
def generate_flashcards():
    data = request.json
    transcript = data.get("transcript")

    # Format the prompt JSON data
    prompt_json = {
        "contents": {
            "role": "user",
            "parts": {
                "text": f'Using the transcript as context, generate 5 flashcards for the given transcript, encase each single transcript in *** symbols. The transcript is {transcript}'
            }
        },
        "generation_config": {
            "temperature": 0.6,
            "topP": 1.0,
            "maxOutputTokens": 5192
        }
    }

    
    # Convert the prompt JSON data to a string
    prompt_json_str = json.dumps(prompt_json)

    # Call the method with the prompt JSON
    model_answer = generate_model_answer(prompt_json_str)

    # Check if the response has the expected structure
    if 'candidates' in model_answer[0] and 'content' in model_answer[0]['candidates'][0] and 'parts' in model_answer[0]['candidates'][0]['content'] and 'text' in model_answer[0]['candidates'][0]['content']['parts'][0]:
        flashcards_text = model_answer[0]['candidates'][0]['content']['parts'][0]['text']
        return jsonify({"flashcards_text": flashcards_text})
    else:
        return jsonify({"error": "Failed to generate flashcards"})

@app.route("/generate_quiz",methods = ["POST"])
def generate_quiz():
    data = request.json
    transcript = data.get("flashcards")

    # Format the prompt JSON data
    prompt_json = {
        "contents": {
            "role": "user",
            "parts": {
                "text": f'Using the flashcards as context, generate 2 quiz questions for the given flashcards, encase each single question in *** symbols. The transcript is {transcript}'
            }
        },
        "generation_config": {
            "temperature": 0.6,
            "topP": 1.0,
            "maxOutputTokens": 5192
        }
    }

    
    # Convert the prompt JSON data to a string
    prompt_json_str = json.dumps(prompt_json)

    # Call the method with the prompt JSON
    model_answer = generate_model_answer(prompt_json_str)

    # Check if the response has the expected structure
    if 'candidates' in model_answer[0] and 'content' in model_answer[0]['candidates'][0] and 'parts' in model_answer[0]['candidates'][0]['content'] and 'text' in model_answer[0]['candidates'][0]['content']['parts'][0]:
        flashcards_text = model_answer[0]['candidates'][0]['content']['parts'][0]['text']
        return jsonify({"flashcards_text": flashcards_text})
    else:
        return jsonify({"error": "Failed to generate flashcards"})
@app.route("/feedback",methods = ["POST"])
def evaluate():
    data = request.json
    questions = data.get("questions")
    answer = data.get("answers")
    # Format the prompt JSON data
    prompt_json = {
        "contents": {
            "role": "user",
            "parts": {
                "text": f'Using the questions as context,give your feedback on the answers submitted as {answer} to the questions {questions}'
            }
        },
        "generation_config": {
            "temperature": 0.6,
            "topP": 1.0,
            "maxOutputTokens": 5192
        }
    }

    
    # Convert the prompt JSON data to a string
    prompt_json_str = json.dumps(prompt_json)

    # Call the method with the prompt JSON
    model_answer = generate_model_answer(prompt_json_str)

    # Check if the response has the expected structure
    if 'candidates' in model_answer[0] and 'content' in model_answer[0]['candidates'][0] and 'parts' in model_answer[0]['candidates'][0]['content'] and 'text' in model_answer[0]['candidates'][0]['content']['parts'][0]:
        flashcards_text = model_answer[0]['candidates'][0]['content']['parts'][0]['text']
        return jsonify({"flashcards_text": flashcards_text})
    else:
        return jsonify({"error": "Failed to generate flashcards"})
if __name__ == "__main__":
    app.run(host="0.0.0.0", port=8000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8000
 * Running on http://192.168.29.151:8000
Press CTRL+C to quit
192.168.29.74 - - [10/Mar/2024 03:45:42] "POST /upload_and_transcribe HTTP/1.1" 308 -


Waiting for server processing...
Waiting for server processing...
Waiting for server processing...
Waiting for server processing...
Waiting for server processing...
Waiting for server processing...
Waiting for server processing...


192.168.29.74 - - [10/Mar/2024 03:46:07] "POST /upload_and_transcribe/ HTTP/1.1" 200 -


Waiting for server processing...
Waiting for server processing...
Waiting for server processing...
Waiting for server processing...
Waiting for server processing...
Waiting for server processing...


192.168.29.74 - - [10/Mar/2024 04:37:19] "POST /upload_and_transcribe/ HTTP/1.1" 200 -
192.168.29.74 - - [10/Mar/2024 04:38:38] "POST /generate_flashcards/ HTTP/1.1" 404 -
192.168.29.74 - - [10/Mar/2024 04:38:55] "POST /generate_flashcards/ HTTP/1.1" 404 -
192.168.29.74 - - [10/Mar/2024 04:38:55] "POST /generate_flashcards/ HTTP/1.1" 404 -
192.168.29.74 - - [10/Mar/2024 04:44:01] "POST /generate_flashcards/ HTTP/1.1" 404 -
192.168.29.74 - - [10/Mar/2024 04:44:02] "POST /generate_flashcards/ HTTP/1.1" 404 -
192.168.29.74 - - [10/Mar/2024 04:44:24] "POST /generate_flashcards/ HTTP/1.1" 404 -
[2024-03-10 04:53:44,127] ERROR in app: Exception on /generate_flashcards [POST]
Traceback (most recent call last):
  File "C:\Users\Fabiha\AppData\Local\Programs\Python\Python310\lib\site-packages\flask\app.py", line 2529, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Fabiha\AppData\Local\Programs\Python\Python310\lib\site-packages\flask\app.py", line 1825, in full_dispatc

In [55]:


# Example prompt JSON data
prompt_json = """
{
  "contents": {
    "role": "user",
    "parts": {
        "text": "Give me an intro about yourself"
    }
  },

  "generation_config": {
    "temperature": 0.6,
    "topP": 1.0,
    "maxOutputTokens": 5192
  }
}
"""

# Call the method with the prompt JSON
model_answer = generate_model_answer(prompt_json)
print(model_answer)

[{'candidates': [{'content': {'role': 'model', 'parts': [{'text': "Greetings! My name is [Your Name], and I'm a highly motivated and results-oriented individual with a passion for [Your Field of Interest]."}]}, 'safetyRatings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'probabilityScore': 0.053206205, 'severity': 'HARM_SEVERITY_NEGLIGIBLE', 'severityScore': 0.059210256}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'probabilityScore': 0.07531231, 'severity': 'HARM_SEVERITY_NEGLIGIBLE', 'severityScore': 0.06348236}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'probabilityScore': 0.053107906, 'severity': 'HARM_SEVERITY_NEGLIGIBLE', 'severityScore': 0.039121646}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'probabilityScore': 0.27973008, 'severity': 'HARM_SEVERITY_NEGLIGIBLE', 'severityScore': 0.12896329}]}]}, {'candidates': [{'content': {'role': 'model', 'parts': [{'te

In [57]:
print(model_answer[0]['candidates'][0]['content']['parts'][0]['text'])

Greetings! My name is [Your Name], and I'm a highly motivated and results-oriented individual with a passion for [Your Field of Interest].
